# FIR design via the transfer function
In this script, I have used the tables of transition values for the filters that are generated.  If the cut-off frequency, or the number of taps is altered, new coefficinets will be required from the tables.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### Generate impulse response
From samples of the frequencey response we can generate the required impulse response.  The parameters are a set of sample points, $\alpha$, and the number of taps.

In [ ]:
def generate_impulse_response(freq_samples, alpha, ir_len):
    # Generate the index of impulse response samples
    n = np.arange(0,ir_len)
    
    # Initialise the returned impulse response
    h = np.zeros(ir_len)
    
    # Now implement the formula that generates the impulse
    # response given the frequency samples

    # The impulse response function depends on the value of alpha
    if alpha==0:
        # Our samples start with a d.c. term
        h = np.ones(ir_len)*freq_samples[0]
        # Now add in all of the other frequency terms
        for idx in range(1,len(freq_samples)):
            h = h + (-1)**idx * freq_samples[idx] * \
                2 * np.cos(np.pi*2*idx*(n+0.5)/ir_len)
    else:
        # There is no d.c. term when alpha=0.5
        for idx in range(0,len(freq_samples)):
            h = h + (-1)**idx * freq_samples[idx] * \
                2 * np.sin(np.pi*(2*idx+1)*(n+0.5)/ir_len)
    h = h/64;

    return(h)

### Plotting Functions
Here we define the plotting functions that we will use for producing the outputs.

In [ ]:
def plot_stem(x, y, name, xlim = [0, 63]):
    """
       Plot FIR frequency design
       INPUT: 
           xlim ([left, right]): set the x limits from let to right. Default: [0, 63].
           x       (array-like): The x-positions of the stems
           y       (array-like): The y-values of the stem heads.
           name        (string): The name used to save figure.
    """
    # Create the plot figure
    plt.figure(figsize=(12, 6))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # stem plot  
    (markerLines, stemLines, baseLines) = plt.stem(x, y, use_line_collection=True)
    plt.setp(baseLines, color = 'black', linewidth=1) 
    plt.setp(stemLines, linewidth=1) 
    markerLines.set_markersize(8)
    markerLines.set_markerfacecolor('none')
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlabel('n')
    plt.ylabel('h(n)')
    plt.xlim(xlim)
    
    # Save figure in python or ipython system
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

In [ ]:
def plot_freq(x, y, x1, y1, name, xlims = [0, 0.5], ylims = [-80, 10]):
    """
       Plot magnitude in dB scale of normalised frequency 
    """
    # Create the plot figure
    plt.figure(figsize=(12, 6))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # Plot the frequency
    ax = plt.gca()
    ax.plot(x, y)
    ax.plot(x1, y1, 's', markerfacecolor = 'none', markersize = 8)
    
    # Tidy up the plot to control axes sizes and labels
    plt.xlabel('Normalised frequency')
    plt.ylabel('Magnitude (dB)')
    plt.xlim(xlims)
    plt.ylim(ylims)
    plt.xticks(np.linspace(0, 0.5, 11))
    
    # Save figure in python or ipython system
    if not is_jupyter():
        plt.tight_layout()
        plt.savefig(name)

### Naïve $\alpha=0$

Here we will define the filter by inverting the samples of the desired transfer function.  In the passband the samples will have a value of 1, and in the stopband a value of 0.  This results in the impulse response consisting of 7 complex exponentials, which simplify to a constant term, plus 3 cosine terms.

In [ ]:
# Generate impulse response
h1 = generate_impulse_response(freq_samples=[1,1,1,1], alpha=0, ir_len=64)

# Zero padd this for plotting
z = np.zeros(4096)
z[0:64] = h1

n = np.arange(0,h1.size)

# Calculate the spacing of the samples so that we can plot them separately on the final plot
spacing = z.size / n.size

# Generate frequency index
f = np.multiply((1/4096), np.arange(0, 4096))

# Calculate the transfer function corresponding to the impulse response
H = 20 * np.log10(abs(np.fft.fft(z)))

Plot the filter impulse response

In [ ]:
plot_stem(x = n, y = h1, name = 'FIR_freq_design_naive_alpha_0.pdf')

Plot the transfer function, along with the original samples that were used to generate the impulse response.

In [ ]:
plot_freq(x = f, y= H, x1 = f[0 : -1 : int(spacing)], y1 = H[0 : -1 : int(spacing)], 
          name = 'FIR_freq_design_naive_alpha_0_transfer.pdf')

### Naïve $\alpha=0.5$

If $\alpha=0.5$, then the first sample is offset from the d.c. value, and we end up with 6 complex exponential terms that result in three sinusoidal terms.

In [ ]:
# Generate impulse response
h2 = generate_impulse_response(freq_samples=[1,1,1], alpha=0.5, ir_len=64)

# Zero padd this for plotting
z[0 : 64] = h2

fft = np.fft.fft(z)
# Avoid zero when calcuating the log of the fft
fft = np.where(fft == 0, 1e-99, fft)

# Convert to dBs
H = 20 * np.log10(abs(fft))

Plot the impulse response

In [ ]:
plot_stem(x = n, y = h2, name = 'FIR_freq_design_naive_alpha_0_5.pdf')

Plot the transfer function

In [ ]:
plot_freq(x = f, y = H, 
          x1 = f[int(spacing/2) : -1 : int(spacing)],
          y1 = H[int(spacing/2) : -1 : int(spacing)], 
          name = 'FIR_freq_design_naive_alpha_0_5_transfer.pdf')

### One transition coefficient with $\alpha=0$ 

This time, an additional term is added to the impusle response for a single transition coefficient beyond the end of the passband.  The coefficient value comes from the transition table for a 64 tap filter.

 BW  |     Minimax  |         T1
 ----|--------------|-----------
  1  | -43.13807279 | 0.42903727
  2  | -42.60259151 | 0.40895721
  3  | -42.69454422 | 0.39915214
  4  | -42.85522858 | 0.39331863
  5  | -43.00406441 | 0.38946941
  6  | -43.12866660 | 0.38676437
  7  | -43.22994751 | 0.38478417
  8  | -43.31122440 | 0.38329559
  9  | -43.37573423 | 0.38215883
 10  | -43.42611473 | 0.38128542
 11  | -43.46438014 | 0.38061717
 12  | -43.49200682 | 0.38011496
 13  | -43.51002055 | 0.37975218
 14  | -43.51906829 | 0.37951108
 15  | -43.51893143 | 0.37938262
 16  | -43.50337124 | 0.37938706
 17  | -43.47679197 | 0.37950305
 18  | -43.43819276 | 0.37973491
 19  | -43.38607107 | 0.38009185
 20  | -43.31829832 | 0.38058889
 21  | -43.23194758 | 0.38124808
 22  | -43.12306250 | 0.38210045
 23  | -42.98644138 | 0.38318832
 24  | -42.81568133 | 0.38456685
 25  | -42.58989504 | 0.38636397
 26  | -42.29372477 | 0.38869300
 27  | -42.12359373 | 0.39071338
 28  | -42.33104649 | 0.39131769
 29  | -43.66271612 | 0.38754643
 30  | -49.58953218 | 0.37016432

In [ ]:
# Generate impulse response
h1 = generate_impulse_response(freq_samples=[1,1,1,1,0.39331863], alpha=0, ir_len=64)

z[0:64] = h1

fft = np.fft.fft(z)
# Avoid zero when calcuating the log of the fft
fft = np.where(fft == 0, 1e-99, fft)

# Convert to dBs
H = 20 * np.log10(abs(fft))

Plot the impulse response

In [ ]:
plot_stem(x = n, y = h1, name = 'FIR_freq_design_transition_alpha_0.pdf')

Plot the transfer function

In [ ]:
plot_freq(x = f, y = H, 
          x1 = f[0 : -1 : int(spacing)], 
          y1 = H[0 : -1 : int(spacing)],
          name = 'FIR_freq_design_transition_alpha_0_transfer.pdf')

### One transition coefficient with $\alpha=0.5$

As before, the filter consists of sine terms, again with one additional term due to the transition coefficient.

 BW  |     Minimax  |         T1
 ----|--------------|-----------
  1  | -53.03119136 | 0.25847710
  2  | -49.90350952 | 0.30463711
  3  | -48.60291028 | 0.32300712
  4  | -47.86102764 | 0.33277230
  5  | -47.33367822 | 0.33893119
  6  | -46.97754010 | 0.34302388
  7  | -46.72567773 | 0.34589374
  8  | -46.54255107 | 0.34797697
  9  | -46.40768757 | 0.34952025
 10  | -46.30861640 | 0.35067226
 11  | -46.23747411 | 0.35152665
 12  | -46.18925238 | 0.35214388
 13  | -46.16084315 | 0.35256314
 14  | -46.15052183 | 0.35280889
 15  | -46.15767925 | 0.35289461
 16  | -46.18272939 | 0.35282465
 17  | -46.22717577 | 0.35259477
 18  | -46.29383352 | 0.35219155
 19  | -46.38729611 | 0.35159035
 20  | -46.51478209 | 0.35075138
 21  | -46.68769185 | 0.34961255
 22  | -46.92458433 | 0.34807636
 23  | -47.14487469 | 0.34637292
 24  | -47.42740283 | 0.34413830
 25  | -47.84619508 | 0.34098168
 26  | -48.51462320 | 0.33629501
 27  | -49.71903979 | 0.32876324
 28  | -52.43790432 | 0.31493401
 29  | -57.56537219 | 0.29131653
 30  | -75.23313068 | 0.23754895

In [ ]:
h2 = generate_impulse_response(freq_samples=[1,1,1,0.32300712], alpha=0.5, ir_len=64)

z[0:64] = h2

fft = np.fft.fft(z)
# Avoid zero when calcuating the log of the fft
fft = np.where(fft == 0, 1e-99, fft)

# Convert to dBs
H = 20 * np.log10(abs(fft))

Plot the impulse response

In [ ]:
plot_stem(x = n, y = h2, name = 'FIR_freq_design_transition_alpha_0_5.pdf')

Plot the transfer function

In [ ]:
plot_freq(x = f, y = H,
          x1 = f[int(spacing/2) : -1 : int(spacing)],
          y1 = H[int(spacing/2) : -1 : int(spacing)],
          name = 'FIR_freq_design_transition_alpha_0_5_transfer.pdf')

### Two transition coefficients with $\alpha=0$

We can add more transition coefficients to improve the stopband rejection.  Here we add two term|s to the original impusle response for two transition coefficients beyond the end of the passband.|

 BW  |     Minimax  |         T1 |         T2
 ----|--------------|------------|-----------
  1  | -71.34605669 | 0.09263874 | 0.58655937
  2  | -68.95100188 | 0.10431198 | 0.59506849
  3  | -67.93254397 | 0.10747244 | 0.59610745
  4  | -67.44900166 | 0.10856629 | 0.59575155
  5  | -67.18725918 | 0.10895954 | 0.59509665
  6  | -67.03460832 | 0.10907286 | 0.59442057
  7  | -66.93190177 | 0.10909093 | 0.59383920
  8  | -66.86171016 | 0.10906313 | 0.59334310
  9  | -66.80449374 | 0.10903805 | 0.59295723
 10  | -66.75452398 | 0.10902604 | 0.59266770
 11  | -66.70635147 | 0.10903719 | 0.59247065
 12  | -66.65580479 | 0.10907898 | 0.59236442
 13  | -66.61315306 | 0.10911932 | 0.59229093
 14  | -66.58374846 | 0.10914004 | 0.59221566
 15  | -66.56856078 | 0.10913574 | 0.59212528
 16  | -66.56996407 | 0.10909761 | 0.59200254
 17  | -66.59197182 | 0.10901270 | 0.59182471
 18  | -66.64072153 | 0.10886269 | 0.59156137
 19  | -66.72532703 | 0.10862172 | 0.59117104
 20  | -66.85935189 | 0.10825315 | 0.59059591
 21  | -67.06340089 | 0.10770412 | 0.58975335
 22  | -67.36988614 | 0.10689648 | 0.58852172
 23  | -67.83240144 | 0.10571086 | 0.58671529
 24  | -68.54578277 | 0.10395716 | 0.58403729
 25  | -69.37429612 | 0.10137252 | 0.57991904
 26  | -70.53978036 | 0.09751009 | 0.57365691
 27  | -72.63886162 | 0.09144898 | 0.56377692
 28  | -77.12408773 | 0.08131032 | 0.54699381
 29  | -90.13150301 | 0.06310367 | 0.51528633

In [ ]:
h1 = generate_impulse_response(freq_samples=[1,1,1,1,0.59575155,0.10856629], alpha=0, ir_len=64)

z[0:64] = h1

fft = np.fft.fft(z)
# Avoid zero when calcuating the log of the fft
fft = np.where(fft == 0, 1e-99, fft)

# Convert to dBs
H = 20 * np.log10(abs(fft))

Plot the transfer function

In [ ]:
plot_freq(x = f, y = H,
          x1 = f[0 : -1 : int(spacing)],
          y1 = H[0 : -1 : int(spacing)],
          name = 'FIR_freq_design_two_transition_alpha_0_transfer.pdf')

### Two transition coefficients with $\alpha=0.5$

As we have done above, we add two terms to the original impulse response expression.

 BW  |     Minimax  |         T1 |         T2
 ----|--------------|------------|-----------
  1  | -81.97820165 | 0.04489532 | 0.39745787
  2  | -77.47677603 | 0.06234297 | 0.47253399
  3  | -75.43366003 | 0.07130853 | 0.50308407
  4  | -74.24422323 | 0.07671668 | 0.51940171
  5  | -73.46603435 | 0.08030149 | 0.52943714
  6  | -72.93950065 | 0.08277505 | 0.53606168
  7  | -72.56980748 | 0.08454181 | 0.54066095
  8  | -72.30567792 | 0.08582789 | 0.54395301
  9  | -72.11738551 | 0.08676782 | 0.54634301
 10  | -71.98698265 | 0.08744490 | 0.54807309
 11  | -71.90354545 | 0.08791162 | 0.54929296
 12  | -71.86074494 | 0.08820051 | 0.55009562
 13  | -71.85560193 | 0.08832996 | 0.55053620
 14  | -71.88797277 | 0.08830718 | 0.55064184
 15  | -71.96055865 | 0.08812921 | 0.55041587
 16  | -72.07948268 | 0.08778219 | 0.54983786
 17  | -72.25485704 | 0.08724091 | 0.54886332
 18  | -72.43054548 | 0.08666751 | 0.54778241
 19  | -72.67739161 | 0.08586042 | 0.54622555
 20  | -73.02645554 | 0.08473955 | 0.54403565
 21  | -73.53427215 | 0.08316908 | 0.54094465
 22  | -74.10930839 | 0.08139764 | 0.53737357
 23  | -74.88277545 | 0.07910531 | 0.53268557
 24  | -76.19851979 | 0.07559324 | 0.52548201
 25  | -77.70372654 | 0.07169698 | 0.51714360
 26  | -80.60026196 | 0.06542802 | 0.50351323
 27  | -84.97097160 | 0.05729977 | 0.48466304
 28  | -97.31820422 | 0.04306546 | 0.44907719
 29  | -127.60738638 | 0.02428725 | 0.39014979

In [ ]:
h2 = generate_impulse_response(freq_samples=[1,1,1,0.50308407,0.07130853], alpha=0.5, ir_len=64)

z[0:64] = h2

fft = np.fft.fft(z)
# Avoid zero when calcuating the log of the fft
fft = np.where(fft == 0, 1e-99, fft)

# Convert to dBs
H = 20 * np.log10(abs(fft))

Plot the transfer function

In [ ]:
plot_freq(x = f, y = H,
          x1 = f[int(spacing/2): -1 : int(spacing)],
          y1 = H[int(spacing/2) : -1 : int(spacing)],
          name = 'FIR_freq_design_two_transition_alpha_0_5_transfer.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.